In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [98]:
#loading the data and filtering clients based on their client_id to match with client df
df = pd.read_csv('df_final_demo.txt')
df_sessions = pd.read_csv("clients_variation.csv", index_col=False).reset_index(drop=True).drop(columns='Unnamed: 0', axis=1)
df_clients = df[df['client_id'].isin(df_sessions['client_id'])].reset_index()

In [99]:
# data cleaning and wrangling
df_sessions.rename(columns={'Variation': 'variation'}, inplace=True)
df_sessions['date_time'] = pd.to_datetime(df_sessions['date_time'], dayfirst=True)
df_sessions['date'] = df_sessions['date_time'].dt.date
df_sessions['time'] = df_sessions['date_time'].dt.time
df_sessions.sort_values(by=['visitor_id', 'visit_id', 'date_time'], inplace=True)
                                                                        

C:\Users\rergu\AppData\Local\Temp\ipykernel_13024\1041178202.py:3: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_sessions['date_time'] = pd.to_datetime(df_sessions['date_time'], dayfirst=True)


In [100]:
#mapping the steps to their respective numbers
steps_dict = {'start':0, 'step_1':1, 'step_2':2, 'step_3':3, 'confirm':4}
df_sessions['step'] = df_sessions['process_step'].map(steps_dict)

In [102]:
# export data to csv for further analysis
df_sessions.to_csv('df_sessions.csv', index=False)

In [4]:
#import ready sessions file...
df_sessions = pd.read_csv('df_sessions.csv')

In [6]:
df_sessions.head()

,client_id,visitor_id,visit_id,process_step,date_time,variation,date,time,step,time_diff
0,6265388,100019786_4272121951,703380671_89376183829_330646,start,2017-04-15 15:13:33,Test,2017-04-15,15:13:33,0,NaN
1,6265388,100019786_4272121951,703380671_89376183829_330646,step_1,2017-04-15 15:13:46,Test,2017-04-15,15:13:46,1,0.216667
2,6265388,100019786_4272121951,703380671_89376183829_330646,step_2,2017-04-15 15:14:21,Test,2017-04-15,15:14:21,2,0.583333
3,6265388,100019786_4272121951,703380671_89376183829_330646,step_3,2017-04-15 15:15:50,Test,2017-04-15,15:15:50,3,1.483333
4,6265388,100019786_4272121951,703380671_89376183829_330646,confirm,2017-04-15 15:16:18,Test,2017-04-15,15:16:18,4,0.466667


In [ ]:
# Step 1: Count how many times each visit_id appears
visit_counts = df_sessions['visit_id'].value_counts()
# Step 2: Identify visit_ids that appear only once
login_only_ids = visit_counts[visit_counts == 1].index
# Step 3: Filter the df_sessions where visit_id is in login_only_ids AND the step is 'start'
login_only_sessions = df_sessions[(df_sessions['visit_id'].isin(login_only_ids)) & 
                                (df_sessions['process_step'] == 'start')]
# Step 4: Count how many such visits exist
login_only_count = login_only_sessions['visit_id'].nunique()
# Step 5: Total number of unique visits
total_visits = df_sessions['visit_id'].nunique()
# Step 6: Compute rate
login_only_rate = login_only_count / total_visits
print(f"Login-only visit rate: {login_only_rate:.2%}")# Step 1: Count how many times each visit_id appears
visit_counts = df_sessions['visit_id'].value_counts()
# Step 2: Identify visit_ids that appear only once
login_only_ids = visit_counts[visit_counts == 1].index
# Step 3: Filter the df_sessions where visit_id is in login_only_ids AND the step is 'start'
login_only_sessions = df_sessions[(df_sessions['visit_id'].isin(login_only_ids)) & 
                                (df_sessions['process_step'] == 'start')]
# Step 4: Count how many such visits exist
login_only_count = login_only_sessions['visit_id'].nunique()
# Step 5: Total number of unique visits
total_visits = df_sessions['visit_id'].nunique()
# Step 6: Compute rate
login_only_rate = login_only_count / total_visits
print(f"Login-only visit rate: {login_only_rate:.2%}")

df_sessions[df_sessions['visit_id'].isin(login_only_ids)]['process_step'].value_counts()

Login-only visit rate: 11.05%
Login-only visit rate: 11.05%


process_step
start      7646
confirm    3142
step_3       52
step_1       42
step_2       28
Name: count, dtype: int64

In [5]:
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321309 entries, 0 to 321308
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   client_id     321309 non-null  int64  
 1   visitor_id    321309 non-null  object 
 2   visit_id      321309 non-null  object 
 3   process_step  321309 non-null  object 
 4   date_time     321309 non-null  object 
 5   variation     321309 non-null  object 
 6   date          321309 non-null  object 
 7   time          321309 non-null  object 
 8   step          321309 non-null  int64  
 9   time_diff     251984 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 24.5+ MB


In [10]:
#date interval is 3 months:
print(df_sessions['date_time'].min())
print(df_sessions['date_time'].max())

2017-03-15 00:19:28
2017-06-20 23:57:06


In [12]:
df_sessions['variation'].value_counts(normalize=True)

variation
Test       0.553508
Control    0.446492
Name: proportion, dtype: float64

In [15]:
#I want to see visitor_ids that have more than 1 visit_id
df_sessions.groupby('visitor_id')['visit_id'].nunique().reset_index().query('visit_id > 1').sort_values(by='visit_id', ascending=False).head(10)

,visitor_id,visit_id
21596,445403329_35219697304,18
44858,819447509_47703321203,14
18201,390393987_61271386429,13
40046,739479428_1803291083,12
49266,890060645_87033036051,12
31076,59863784_32003824003,12
23529,476952653_45563039199,12
19388,409434116_1011445691,11
38593,716747126_56844389124,11
54775,97982744_40889261423,10


In [78]:
#check some customers with more than 1 visit_id
df_sessions[df_sessions['client_id'] == 9008485].sort_values(by='visit_id').head(10)

,client_id,visitor_id,visit_id,process_step,date_time,variation,date,time,time_diff,step
286587,9008485,901570806_56331117092,127828451_24580384403_991317,start,2017-04-12 13:46:53,Control,2017-04-12,13:46:53,0.233333,0
286586,9008485,901570806_56331117092,127828451_24580384403_991317,start,2017-04-12 13:46:39,Control,2017-04-12,13:46:39,NaN,0
51334,9008485,24155384_37786931031,222653209_61587850178_557903,start,2017-04-13 01:23:56,Control,2017-04-13,01:23:56,NaN,0
51335,9008485,24155384_37786931031,222653209_61587850178_557903,start,2017-04-13 01:23:57,Control,2017-04-13,01:23:57,0.016667,0
222151,9008485,720611138_67735334882,240615538_86110374999_405398,start,2017-04-12 16:17:12,Control,2017-04-12,16:17:12,NaN,0
222152,9008485,720611138_67735334882,240615538_86110374999_405398,start,2017-04-12 16:17:14,Control,2017-04-12,16:17:14,0.033333,0
44013,9008485,220817442_76661647214,366896661_55080385620_622929,start,2017-04-12 13:54:16,Control,2017-04-12,13:54:16,NaN,0
44014,9008485,220817442_76661647214,366896661_55080385620_622929,start,2017-04-12 13:54:21,Control,2017-04-12,13:54:21,0.083333,0
128799,9008485,460913705_72189240051,380585873_81379523466_353718,start,2017-04-13 16:34:26,Control,2017-04-13,16:34:26,6.800000,0
128798,9008485,460913705_72189240051,380585873_81379523466_353718,start,2017-04-13 16:27:38,Control,2017-04-13,16:27:38,0.083333,0


In [17]:
# I want to see client_ids per visitor_id more than 1
df_sessions.groupby('visitor_id')['client_id'].nunique().reset_index().query('client_id > 1').sort_values(by='client_id', ascending=False)


,visitor_id,client_id
34474,652206951_94195881749,4
41,100636834_70620124872,2
36777,6881546_427030029,2
38239,711208998_14643001925,2
38205,710765917_90350210282,2
...,...,...
18457,394247857_56670565004,2
18155,389718065_59253104368,2
17953,386546813_81776454991,2
17662,381789213_30254245920,2


In [ ]:
### AB Testing KPI's
# 1. Conversion Rate
# 2. Step Drop-off Rate
# 3. Steps Per Session
# 4. Bounce Rate
# 5. Looping Rate
# 6. Funnel Completion Rate
# 7. Average Time Per Step
# 8. Average Time Between Steps
# 9. Average Time Per Session
# 10. Average Time Per Visitor
# 11. Average Time to Conversion


In [325]:
#1 Conversion Rate:	% of sessions that reach step_4 (confirmation step)
df_grouped = df_sessions.groupby(['variation', 'step'])['visit_id'].nunique().reset_index()\
    .rename(columns= {'visit_id': 'session_count'})
total_sessions = df_sessions.groupby(['variation'])['visit_id'].nunique().reset_index().rename(columns= {'visit_id': 'unique_sessions'})
df_conversion_rate = df_grouped.merge(total_sessions, on='variation', how='left')
df_conversion_rate['success_rate'] = df_conversion_rate['session_count']/df_conversion_rate['unique_sessions']
df_conversion_rate = round(df_conversion_rate[df_conversion_rate['step'] == 4],2)
df_conversion_rate
conversion_rate = df_conversion_rate[['variation', 'success_rate']].reset_index(drop=True)  
conversion_rate


,variation,success_rate
0,Control,0.50
1,Test,0.59


In [ ]:
#1.a conversion rate following the steps:
''' Once we have this table, we can:
Run a Z-test or Chi-squared test for proportions to assess statistical significance.
Interpret effect size (how big is the lift or drop).
Optionally, calculate confidence intervals for the conversion rate'''

In [400]:
#2 Step drop-off rate: % of sessions that drop off at each step

# Count how many unique sessions exist per variation
total_sessions = df_sessions.groupby('variation')['visit_id'].nunique().reset_index(name='total_sessions')

# Count how many reach each step
df_step_counts = df_sessions.groupby(['variation', 'step'])['visit_id'].nunique().reset_index()
df_step_counts = df_step_counts.rename(columns={'visit_id': 'session_count'})

# Merge total session counts with step 0
df_step_counts = df_step_counts.merge(total_sessions, on='variation', how='left')

# Create prev_step_count column
df_step_counts['prev_step_count'] = df_step_counts.groupby('variation')['session_count'].shift(1)

# For step 0, use total_sessions as the baseline for drop-in
df_step_counts.loc[df_step_counts['step'] == 0, 'prev_step_count'] = df_step_counts.loc[df_step_counts['step'] == 0, 'total_sessions']

# Calculate drop ratio
df_step_counts['step_drop_ratio'] = round(1 - df_step_counts['session_count'] / df_step_counts['prev_step_count'], 2)

# Final output
step_drop_rate = df_step_counts[['variation', 'step', 'step_drop_ratio']]

step_drop_rate





,variation,step,step_drop_ratio
0,Control,0,0.04
1,Control,1,0.24
2,Control,2,0.14
3,Control,3,0.09
4,Control,4,0.12
5,Test,0,0.11
6,Test,1,0.15
7,Test,2,0.13
8,Test,3,0.09
9,Test,4,0.02


In [ ]:
# 2.x Average drop-off rate per variation
avg_drop_rate = step_drop_rate.groupby('variation')['step_drop_ratio'].mean().reset_index()
avg_drop_rate.rename(columns={'step_drop_ratio': 'avg_step_drop_ratio'}, inplace=True) 
avg_drop_rate

,variation,avg_step_drop_ratio
0,Control,0.126
1,Test,0.100


In [259]:
#2.a dropoff rate following the steps:
'''You can use this to:
Visualize drop-off curves (e.g., funnel charts).
Compare drop-off rates per step using chi-squared tests (are more users getting stuck in one group?).
Combine this with time per step to investigate if users are hesitant or confused. '''

'You can use this to:\nVisualize drop-off curves (e.g., funnel charts).\nCompare drop-off rates per step using chi-squared tests (are more users getting stuck in one group?).\nCombine this with time per step to investigate if users are hesitant or confused. '

In [316]:
#3 steps per session on average:

df_steps = df_sessions.groupby('variation').agg(total_sessions=pd.NamedAgg(column='visit_id', aggfunc='count'),
                                                unique_sessions=pd.NamedAgg(column='visit_id', aggfunc='nunique'))\
                                                    .reset_index()
df_steps['avg_steps'] = round(df_steps['total_sessions']/df_steps['unique_sessions'],2)
avg_steps = df_steps[['variation','avg_steps']]
avg_steps

,variation,avg_steps
0,Control,4.46
1,Test,4.79


In [ ]:
#3.a following steps:
'''Look for:
Lower average steps in the test group (if the goal is a cleaner journey)
Higher average steps with higher conversion may signal productive engagement
Higher average steps with lower conversion could indicate friction or confusion.'''

In [317]:
#4 bounce rate: the percentage of sessions that only visit the first step (start) and do not proceed to any other step in the funnel.
# step 0 is the start..

# Find sessions (visit_id) where only step 0 is present
first_step_sessions = df_sessions[df_sessions['step'] == 0].groupby(['variation', 'visit_id']).size().reset_index(name='count')
# Now check if these sessions have ONLY step 0 (i.e., visit_id appears only once in the whole df_sessions)
session_step_counts = df_sessions.groupby(['variation', 'visit_id'])['step'].nunique().reset_index()
bounce_sessions = session_step_counts[session_step_counts['step'] == 1]
# Filter only those where the only step is 0
bounce_sessions = bounce_sessions.merge(df_sessions[df_sessions['step'] == 0][['variation', 'visit_id']], on=['variation', 'visit_id']).drop_duplicates()

df_bounce = bounce_sessions.groupby('variation')['step'].sum().reset_index(name='count_step_0')

df_bounce
bounce_summary = pd.merge(left=df_steps, right=df_bounce, on='variation', how='left')
bounce_summary['bounce_rate'] = round(bounce_summary['count_step_0']/ bounce_summary['total_sessions'],3)

# show only bounce_rate from bounce_summary
bounce_rate = bounce_summary[['variation', 'bounce_rate']]
bounce_rate

,variation,bounce_rate
0,Control,0.053
1,Test,0.029


In [ ]:
#4.a following steps:
'''What Comes Next:
Compare bounce rates between control and test groups.
If bounce rate improves (decreases) in the test group, it’s an early sign of better engagement.
Use a chi-squared test to statistically compare the proportions.
If bounce is high in both groups, further investigate landing page UX, loading speed, or targeting.
Analyze session recordings or feedback for sessions that bounced.
'''

In [318]:
#5 revisit behaviours (looping rate): % of sessions where steps are revisited (e.g., step_1 → start → step_1)
# Indicates confusion, comparison, or lack of confidence

step_counts = df_sessions.groupby(['variation', 'visit_id', 'step']).size().reset_index(name='step_visits')

loop_sessions = step_counts.groupby(['variation', 'visit_id'])['step_visits'].max().reset_index()

loop_sessions['is_loop'] = loop_sessions['step_visits'] > 1

total_sessions = loop_sessions.groupby('variation')['visit_id'].nunique().reset_index(name='total_sessions')
looping_sessions = loop_sessions[loop_sessions['is_loop']].groupby('variation')['visit_id'].nunique().reset_index(name='loop_sessions')

looping_summary = pd.merge(total_sessions, looping_sessions, on='variation', how='left').fillna(0)
looping_summary['looping_rate'] = round(looping_summary['loop_sessions'] / looping_summary['total_sessions'],2)

looping_rate = looping_summary[['variation', 'looping_rate']]
looping_rate


,variation,looping_rate
0,Control,0.41
1,Test,0.46


In [ ]:
#5.a following steps:
''' What Comes Next:
Calculate looping rate for each variation (sessions where any step is visited more than once).
Compare looping rates between Test and Control.
High looping in the Test group could signal UX problems, while lower looping may indicate a smoother flow.
Use a chi-squared test to determine if the difference in looping rates is statistically significant.
Deep-dive into session paths with looping for qualitative review (e.g., session replays or path visualizations).'''

In [329]:
#6 Funnel Completion Rate: % of sessions completing all funnel steps sequentially
# Measures how well the entire flow performs


required_steps = set(range(5))  # Steps 0 to 4

# For each session, collect all unique steps visited
session_steps = df_sessions.groupby(['variation', 'visit_id'])['step'].apply(set).reset_index(name='steps_visited')

# Check if all required steps are present
session_steps['completed_funnel'] = session_steps['steps_visited'].apply(lambda steps: required_steps.issubset(steps))

# Calculate completed funnels per variation
completed = session_steps.groupby('variation')['completed_funnel'].sum().reset_index(name='completed_sessions')

# Calculate total sessions per variation
total = session_steps.groupby('variation')['visit_id'].nunique().reset_index(name='total_sessions')

# Merge and compute completion rate
funnel_completion = pd.merge(total, completed, on='variation')
funnel_completion['funnel_completion_rate'] = round(funnel_completion['completed_sessions'] / funnel_completion['total_sessions'],2)

funnel_completion_rate = funnel_completion[['variation', 'funnel_completion_rate']]
funnel_completion_rate



,variation,funnel_completion_rate
0,Control,0.46
1,Test,0.48


In [ ]:
#6.a following steps:
''' What Comes Next:
Compare funnel completion rates between Test and Control groups.
Use a chi-squared test or z-test for proportions to assess statistical significance of any difference.
Combine with drop-off and looping rates to diagnose why one group may outperform the other.
If completion is low in both groups, analyze which step(s) lose the most users.
Use insights to recommend A/B variant improvements or additional testing.'''

In [ ]:
# 7. Average Time Per Step
# 8. Average Time Between Steps
# 9. Average Time Per Session
# 10. Total Time to Conversion


In [ ]:
# 7. Average Time Per Step
# For each step in each session, calculate time spent before moving to the next step but step 4 is the confirmation step so there is no next step
def calculate_time_per_step(df):
    df['next_step_time'] = df.groupby(['variation', 'visit_id'])['date_time'].shift(-1)
    df['time_spent'] = (df['next_step_time'] - df['date_time']).dt.total_seconds() / 60  # Convert to minutes
    # Exclude confirmation step
    avg_time_per_step = df.groupby(['variation', 'step'])['time_spent'].mean().reset_index()
    avg_time_per_step.rename(columns={'time_spent': 'avg_time_per_step'}, inplace=True)
    return avg_time_per_step
     
calculate_time_per_step(df_sessions)


,variation,step,avg_time_per_step
0,Control,0,1.048015
1,Control,1,0.837717
2,Control,2,1.522998
3,Control,3,2.269543
4,Control,4,2.549970
5,Test,0,1.009512
6,Test,1,1.008972
7,Test,2,1.476736
8,Test,3,2.151944
9,Test,4,3.957500


In [394]:
# 8. Average Time in per Step
# based on the above function, we can calculate the average time per step for each variation exclude step 4 (confirmation step) by groupping per variation and step

avg_time_per_step = calculate_time_per_step(df_sessions)
avg_time_per_step = avg_time_per_step[avg_time_per_step['step'] != 4].groupby('variation')['avg_time_per_step'].mean().reset_index()
avg_time_per_step
# Add a new column to the avg_time_per_step dataframe for step 4
step_4_avg_time = calculate_time_per_step(df_sessions[df_sessions['step'] == 4])
step_4_avg_time = step_4_avg_time.groupby('variation')['avg_time_per_step'].mean().reset_index()
step_4_avg_time.rename(columns={'avg_time_per_step': 'avg_time_step_4'}, inplace=True)

# Merge step 4 average time with avg_time_per_step
avg_time_per_step = avg_time_per_step.merge(step_4_avg_time, on='variation', how='left')
avg_time_per_step.rename(columns={'avg_time_per_step': 'avg_time_per_step_excl_step4'}, inplace=True)
avg_time_per_step

# add a new column to the avg_time_per_step dataframe including step 4
avg_time_per_step['avg_time_per_step_incl_step4'] = avg_time_per_step['avg_time_per_step_excl_step4'] + avg_time_per_step['avg_time_step_4']
# rename the columns
avg_time_per_step = avg_time_per_step[['variation', 'avg_time_per_step_excl_step4', 'avg_time_step_4', 'avg_time_per_step_incl_step4']]
avg_time_per_step


C:\Users\rergu\AppData\Local\Temp\ipykernel_13024\2012628424.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_step_time'] = df.groupby(['variation', 'visit_id'])['date_time'].shift(-1)
C:\Users\rergu\AppData\Local\Temp\ipykernel_13024\2012628424.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_spent'] = (df['next_step_time'] - df['date_time']).dt.total_seconds() / 60  # Convert to minutes


,variation,avg_time_per_step_excl_step4,avg_time_step_4,avg_time_per_step_incl_step4
0,Control,1.419569,2.562337,3.981906
1,Test,1.411791,4.101090,5.512881


In [15]:
df_sessions[df_sessions['step']==4].head(20)

,client_id,visitor_id,visit_id,process_step,date_time,variation,date,time,step,time_diff
4,6265388,100019786_4272121951,703380671_89376183829_330646,confirm,2017-04-15 15:16:18,Test,2017-04-15,15:16:18,4,0.466667
5,6265388,100019786_4272121951,703380671_89376183829_330646,confirm,2017-04-15 15:16:47,Test,2017-04-15,15:16:47,4,0.483333
14,2340487,100026388_52569174348,560716384_55516231568_734638,confirm,2017-06-09 18:41:43,Test,2017-06-09,18:41:43,4,0.816667
28,762846,10008560_69050398598,312355704_26376649868_97900,confirm,2017-04-15 22:56:27,Control,2017-04-15,22:56:27,4,0.616667
38,5665278,100120667_10952149000,229897790_94515565093_1603,confirm,2017-04-09 16:35:46,Control,2017-04-09,16:35:46,4,0.733333
39,1187711,100138619_81599280536,635883778_37858314558_285550,confirm,2017-06-04 14:28:01,Test,2017-06-04,14:28:01,4,NaN
55,6230585,100189756_32488226026,671811152_97876326241_999687,confirm,2017-05-08 09:29:38,Test,2017-05-08,09:29:38,4,0.900000
76,8578422,100206138_44165664343,407425819_96032420747_134542,confirm,2017-04-07 20:18:34,Control,2017-04-07,20:18:34,4,1.016667
86,1870359,100214674_18719887389,582615521_62348187689_764378,confirm,2017-04-06 20:41:39,Control,2017-04-06,20:41:39,4,2.816667
101,6099993,100265469_84689631452,263946645_78836270528_169106,confirm,2017-03-29 16:27:04,Test,2017-03-29,16:27:04,4,0.483333


In [14]:
df_sessions[df_sessions['step']==4].groupby('visit_id').nunique().reset_index().query('date_time > 1')

,visit_id,client_id,visitor_id,process_step,date_time,variation,date,time,step,time_diff
0,100012776_37918976071_457913,1,1,1,2,1,1,2,1,1
8,100258507_71262593004_214494,1,1,1,3,1,1,3,1,3
24,100751264_35071231525_844320,1,1,1,4,1,1,4,1,4
28,100844233_60058998075_722846,1,1,1,2,1,1,2,1,2
42,101351075_24045289214_506540,1,1,1,2,1,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...
37629,999066804_34223271414_790731,1,1,1,2,1,1,2,1,2
37657,999522487_18357180783_393962,1,1,1,2,1,1,2,1,2
37671,999841812_92641758733_294220,1,1,1,2,1,1,2,1,1
37674,999954858_74676709104_879685,1,1,1,2,1,1,2,1,2


In [ ]:
# 9. Average Time Per Session based on session duration
# Calculate session duration as the difference between the first and last date_time for each session
session_times = df_sessions.groupby(['variation', 'visit_id'])['date_time'].agg(['min', 'max']).reset_index()
session_times['session_duration'] = (session_times['max'] - session_times['min']).dt.total_seconds() / 60.0  # duration in minutes

# Now average per variation
average_time_per_session = session_times.groupby('variation')['session_duration'].mean().reset_index(name='average_time_per_session')
average_time_per_session

,variation,average_time_per_session
0,Control,4.693355
1,Test,5.279631


In [382]:
# 10. Total Time to Conversion which is the time from the first step to the confirmation step
conversion_times = df_sessions[df_sessions['step'] == 4].groupby(['variation', 'visit_id'])['date_time'].min().reset_index()    
conversion_times = conversion_times.rename(columns={'date_time': 'conversion_time'})
first_steps = df_sessions[df_sessions['step'] == 0].groupby(['variation', 'visit_id'])['date_time'].min().reset_index()
first_steps = first_steps.rename(columns={'date_time': 'first_step_time'}) 
# Merge to get the time to conversion
conversion_summary = pd.merge(first_steps, conversion_times, on=['variation', 'visit_id'])
conversion_summary['time_to_conversion'] = (conversion_summary['conversion_time'] - conversion_summary['first_step_time']).dt.total_seconds() / 60.0  # duration in minutes
# Now average per variation
average_time_to_conversion = conversion_summary.groupby('variation')['time_to_conversion'].mean().reset_index(name='average_time_to_conversion')
average_time_to_conversion


,variation,average_time_to_conversion
0,Control,6.513386
1,Test,6.235432


In [ ]:
#following steps:
'''What Comes Next:
Compare timing metrics for Test vs. Control groups.

Statistical testing: Use t-tests or Mann-Whitney U tests to see if timing differences are significant.

Visualize: Plot time distributions with boxplots or histograms for each variation.

Investigate: Look at sessions or steps with unusually long or short times for UX improvement opportunities.'''